In [32]:
import zipfile

In [20]:
# Cell 1: Setup
import google.generativeai as genai
import pandas as pd
import json
from google.colab import files
import os

# Configure Gemini
API_KEY = os.getenv('GEMINI_API_KEY')  # Get from environment variable
if not API_KEY:
    # Fallback to Colab secrets if environment variable is not set
    from google.colab import userdata
    try:
        API_KEY = userdata.get('GEMINI_API_KEY')
    except Exception as e:
        print(f"Could not get API key from Colab secrets: {e}")
        API_KEY = None

if not API_KEY:
    print("🔑 Gemini API key not found. Please set the GEMINI_API_KEY environment variable or add it to Colab secrets.")
else:
    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel('gemini-flash-latest')
    print("✅ Setup complete! Ready to merge data.")

✅ Setup complete! Ready to merge data.


In [21]:
# Cell to test Gemini API
try:
    test_prompt = "Write a short, creative sentence."
    response = model.generate_content(test_prompt)
    print("Gemini test successful!")
    print(f"Response: {response.text}")
except Exception as e:
    print(f"Gemini test failed: {e}")

Gemini test successful!
Response: The teapot dreamed of a tidal wave.


In [25]:
# Cell 2: Upload Multiple Files (Bundle 1 & Bundle 2)
print("📤 Upload ALL Bundle 1 files (CSV/Excel) - select multiple files:")
bundle1_files = files.upload()

print("\n📤 Upload ALL Bundle 2 files (CSV/Excel) - select multiple files:")
bundle2_files = files.upload()

print(f"\n✅ Uploaded:")
print(f"   Bundle 1: {len(bundle1_files)} files")
print(f"   Bundle 2: {len(bundle2_files)} files")

📤 Upload ALL Bundle 1 files (CSV/Excel) - select multiple files:


Saving Bank1_Mock_Loan_Transactions.csv to Bank1_Mock_Loan_Transactions (2).csv
Saving Bank1_Mock_Loan_Accounts.xlsx to Bank1_Mock_Loan_Accounts (2).xlsx
Saving Bank1_Mock_FixedTerm_Transactions.csv to Bank1_Mock_FixedTerm_Transactions (2).csv
Saving Bank1_Mock_FixedTerm_Accounts.xlsx to Bank1_Mock_FixedTerm_Accounts (2).xlsx
Saving Bank1_Mock_Customer.xlsx to Bank1_Mock_Customer (2).xlsx
Saving Bank1_Mock_CurSav_Transactions.csv to Bank1_Mock_CurSav_Transactions (2).csv
Saving Bank1_Mock_CurSav_Accounts.xlsx to Bank1_Mock_CurSav_Accounts (2).xlsx

📤 Upload ALL Bundle 2 files (CSV/Excel) - select multiple files:


Saving Bank2_Mock_Loan_Transactions.xlsx to Bank2_Mock_Loan_Transactions (2).xlsx
Saving Bank2_Mock_Loan_Accounts.xlsx to Bank2_Mock_Loan_Accounts (2).xlsx
Saving Bank2_Mock_Identifications.xlsx to Bank2_Mock_Identifications (2).xlsx
Saving Bank2_Mock_Deposit_Transactions.xlsx to Bank2_Mock_Deposit_Transactions (2).xlsx
Saving Bank2_Mock_Deposit_Accounts.xlsx to Bank2_Mock_Deposit_Accounts (2).xlsx
Saving Bank2_Mock_Customer.xlsx to Bank2_Mock_Customer (2).xlsx
Saving Bank2_Mock_Addresses.xlsx to Bank2_Mock_Addresses (2).xlsx

✅ Uploaded:
   Bundle 1: 7 files
   Bundle 2: 7 files


In [26]:
# Cell 3: Load All Tables from Both Bundles
def load_tables(file_dict, bundle_name):
    """Load all CSV/Excel files into dictionary of DataFrames"""
    tables = {}
    for filename in file_dict.keys():
        try:
            if filename.endswith('.csv'):
                df = pd.read_csv(filename)
            elif filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(filename)
            else:
                continue

            # Use filename (without extension) as table name
            table_name = filename.rsplit('.', 1)[0]
            tables[table_name] = df
            print(f"  ✓ {table_name}: {len(df)} rows, {len(df.columns)} columns")
        except Exception as e:
            print(f"  ✗ Error loading {filename}: {e}")

    return tables

print("📊 Loading Bundle 1 tables:")
bundle1_tables = load_tables(bundle1_files, "Bundle 1")

print("\n📊 Loading Bundle 2 tables:")
bundle2_tables = load_tables(bundle2_files, "Bundle 2")

📊 Loading Bundle 1 tables:
  ✓ Bank1_Mock_Loan_Transactions (2): 73313 rows, 16 columns
  ✓ Bank1_Mock_Loan_Accounts (2): 14729 rows, 31 columns
  ✓ Bank1_Mock_FixedTerm_Transactions (2): 75265 rows, 17 columns
  ✓ Bank1_Mock_FixedTerm_Accounts (2): 14969 rows, 20 columns
  ✓ Bank1_Mock_Customer (2): 10000 rows, 24 columns
  ✓ Bank1_Mock_CurSav_Transactions (2): 74604 rows, 23 columns
  ✓ Bank1_Mock_CurSav_Accounts (2): 15071 rows, 16 columns

📊 Loading Bundle 2 tables:
  ✓ Bank2_Mock_Loan_Transactions (2): 74921 rows, 18 columns
  ✓ Bank2_Mock_Loan_Accounts (2): 14986 rows, 33 columns
  ✓ Bank2_Mock_Identifications (2): 10000 rows, 8 columns
  ✓ Bank2_Mock_Deposit_Transactions (2): 99147 rows, 26 columns
  ✓ Bank2_Mock_Deposit_Accounts (2): 19756 rows, 25 columns
  ✓ Bank2_Mock_Customer (2): 10000 rows, 16 columns
  ✓ Bank2_Mock_Addresses (2): 10000 rows, 7 columns


In [29]:
# Cell 4: AI-Powered Schema Analysis & Mapping
def analyze_all_schemas(bundle1_tables, bundle2_tables):
    """Get comprehensive schema for all tables"""

    def convert_timestamps_to_strings(obj):
        """Recursively convert Timestamp objects to strings in a dictionary or list."""
        if isinstance(obj, dict):
            return {k: convert_timestamps_to_strings(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_timestamps_to_strings(elem) for elem in obj]
        elif isinstance(obj, pd.Timestamp):
            return str(obj)
        else:
            return obj

    # Build schema summary for Bundle 1
    b1_schema = {}
    for table_name, df in bundle1_tables.items():
        b1_schema[table_name] = {
            "columns": list(df.columns),
            "sample": convert_timestamps_to_strings(df.head(2).to_dict('records')),
            "row_count": len(df)
        }

    # Build schema summary for Bundle 2
    b2_schema = {}
    for table_name, df in bundle2_tables.items():
        b2_schema[table_name] = {
            "columns": list(df.columns),
            "sample": convert_timestamps_to_strings(df.head(2).to_dict('records')),
            "row_count": len(df)
        }

    return b1_schema, b2_schema

def generate_mappings(b1_schema, b2_schema):
    """Use Gemini to create intelligent mappings"""

    prompt = f"""
    You are a data integration expert. Analyze these two data bundles and create mappings.

    BUNDLE 1 SCHEMA:
    {json.dumps(b1_schema, indent=2)}

    BUNDLE 2 SCHEMA:
    {json.dumps(b2_schema, indent=2)}

    TASK: Create field-level mappings from Bundle 1 to Bundle 2.

    RULES:
    - One Bundle 1 table can map to MULTIPLE Bundle 2 tables (1-to-many)
    - Multiple Bundle 1 tables can map to ONE Bundle 2 table (many-to-1)
    - Map based on semantic meaning, not just column names
    - For unmapped columns in Bundle 1, suggest which Bundle 2 table should receive them

    Return JSON in this EXACT format:
    {{
      "table_mappings": [
        {{
          "source_table": "bundle1_table_name",
          "target_table": "bundle2_table_name",
          "field_mappings": [
            {{
              "source_field": "column_from_bundle1",
              "target_field": "column_in_bundle2",
              "confidence": 0.95,
              "reasoning": "why this mapping makes sense"
            }}
          ]
        }}
      ],
      "summary": "brief explanation of mapping strategy"
    }}
    """

    print("🤖 Asking Gemini to analyze schemas and create mappings...")
    response = model.generate_content(prompt)

    # Extract JSON from response
    response_text = response.text
    json_start = response_text.find('{')
    json_end = response_text.rfind('}') + 1
    json_string = response_text[json_start:json_end]

    mappings = json.loads(json_string)
    return mappings

# Run analysis
b1_schema, b2_schema = analyze_all_schemas(bundle1_tables, bundle2_tables)
mappings = generate_mappings(b1_schema, b2_schema)

print("\n✅ Mapping Analysis Complete!")
print(f"\n📋 Strategy: {mappings.get('summary', 'N/A')}")
print(f"\n🔗 Found {len(mappings['table_mappings'])} table-level mappings")

🤖 Asking Gemini to analyze schemas and create mappings...
This analysis integrates banking data from Bundle 1 (segmented by product type: Loan, Fixed Term, Current/Savings) into the generalized structures of Bundle 2 (segmented by Loan and Deposit).

## Data Integration Mappings (Bundle 1 to Bundle 2)

### Summary

The mapping strategy focuses on normalizing three main domains: Customers, Loan Accounts/Transactions, and Deposit Accounts/Transactions. Bundle 1's Customer table is split across three tables in Bundle 2 (`Customer`, `Addresses`, `Identifications`). Bundle 1's Fixed Term and Cur/Sav structures are consolidated into Bundle 2's general `Deposit` structures. Key IDs (UUIDs) in Bundle 1 (`accountId`, `transactionReference`, `customerId`) are mapped to the appropriate primary or foreign key fields in Bundle 2 (`encodedKey`, `id`, `parentAccountKey`, `accountHolderKey`).

---

### Table Mappings

```json
{
  "table_mappings": [
    {
      "source_table": "Bank1_Mock_Customer (2)

In [30]:
# Cell 5: Apply Transformations & Merge
def apply_mappings(bundle1_tables, bundle2_tables, mappings):
    """Transform Bundle 1 data and merge into Bundle 2"""

    merged_tables = {}

    # Start with copies of Bundle 2 tables
    for table_name, df in bundle2_tables.items():
        merged_tables[table_name] = df.copy()

    # Process each table mapping
    for mapping in mappings['table_mappings']:
        source_table = mapping['source_table']
        target_table = mapping['target_table']
        field_mappings = mapping['field_mappings']

        if source_table not in bundle1_tables:
            print(f"⚠️  Source table '{source_table}' not found, skipping...")
            continue

        if target_table not in merged_tables:
            print(f"⚠️  Target table '{target_table}' not found, creating new...")
            merged_tables[target_table] = pd.DataFrame()

        print(f"\n🔄 Mapping: {source_table} → {target_table}")

        # Transform source data
        source_df = bundle1_tables[source_table]
        transformed = pd.DataFrame()

        for field_map in field_mappings:
            source_field = field_map['source_field']
            target_field = field_map['target_field']

            if source_field in source_df.columns:
                transformed[target_field] = source_df[source_field]
                print(f"  ✓ {source_field} → {target_field} ({field_map.get('confidence', 'N/A')})")

        # Add missing columns from target with null values
        target_df = merged_tables[target_table]
        for col in target_df.columns:
            if col not in transformed.columns:
                transformed[col] = None

        # Reorder to match target schema
        if len(target_df.columns) > 0:
            transformed = transformed[target_df.columns]

        # Merge
        merged_tables[target_table] = pd.concat(
            [target_df, transformed],
            ignore_index=True
        )

        print(f"  📊 Added {len(transformed)} rows to {target_table}")

    return merged_tables

# Apply all mappings
print("=" * 60)
merged_tables = apply_mappings(bundle1_tables, bundle2_tables, mappings)

print("\n" + "=" * 60)
print("✅ MERGE COMPLETE!")
print("\n📊 Final Table Sizes:")
for table_name, df in merged_tables.items():
    original_size = len(bundle2_tables.get(table_name, pd.DataFrame()))
    added = len(df) - original_size
    print(f"  {table_name}: {len(df)} rows (+{added} from Bundle 1)")



🔄 Mapping: Bank1_Mock_Customer (2) → Bank2_Mock_Customer (2)
  ✓ customerId → encodedKey (0.95)
  ✓ givenName → firstName (1.0)
  ✓ lastName → lastName (1.0)
  ✓ dateOfBirth → birthDate (1.0)
  ✓ gender → gender (1.0)
  ✓ email → emailAddress (1.0)
  ✓ phoneNumber → homePhone (0.8)
  ✓ smsNumber → mobilePhone (0.9)
  ✓ language → preferredLanguage (0.9)
  ✓ customerStatus → state (0.9)
  ✓ customerType → clientType (0.9)
  ✓ accountOfficerId → assignedUserKey (0.9)
  📊 Added 10000 rows to Bank2_Mock_Customer (2)

🔄 Mapping: Bank1_Mock_Customer (2) → Bank2_Mock_Addresses (2)
  ✓ customerId → parentKey (1.0)
  ✓ street → line1 (1.0)
  ✓ addressCity → city (1.0)
  ✓ state → region (1.0)
  ✓ postCode → postcode (1.0)
  ✓ country → country (1.0)
  📊 Added 10000 rows to Bank2_Mock_Addresses (2)

🔄 Mapping: Bank1_Mock_Customer (2) → Bank2_Mock_Identifications (2)
  ✓ customerId → clientKey (1.0)
  ✓ legalId → documentId (1.0)
  ✓ legalDocumentName → documentType (1.0)
  ✓ legalIssueAuthorise

/tmp/ipython-input-1638667000.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_tables[target_table] = pd.concat(


  📊 Added 14729 rows to Bank2_Mock_Loan_Accounts (2)

🔄 Mapping: Bank1_Mock_Loan_Transactions (2) → Bank2_Mock_Loan_Transactions (2)
  ✓ transactionReference → encodedKey (0.95)
  ✓ activity → type (1.0)
  ✓ transactionAmount → amount (1.0)
  ✓ currency → originalCurrencyCode (1.0)
  ✓ effectiveDate → valueDate (1.0)
  ✓ reason → notes (0.8)
  ✓ branch → branchKey (1.0)
  ✓ principalAmount → principalAmount (1.0)
  ✓ interestAmount → interestAmount (1.0)
  ✓ chargeAmount → feesAmount (1.0)
  ✓ balanceAmount → totalBalance (0.9)
  ✓ interestRate → interestRate (1.0)
  ✓ channelName → transactionChannel (0.9)
  ✓ accountId → parentAccountKey (1.0)


/tmp/ipython-input-1638667000.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_tables[target_table] = pd.concat(
/tmp/ipython-input-1638667000.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_tables[target_table] = pd.concat(


  📊 Added 73313 rows to Bank2_Mock_Loan_Transactions (2)

🔄 Mapping: Bank1_Mock_FixedTerm_Accounts (2) → Bank2_Mock_Deposit_Accounts (2)
  ✓ accountId → encodedKey (0.95)
  ✓ accountTitles → name (0.9)
  ✓ productId → productTypeKey (1.0)
  ✓ currency → currencyCode (1.0)
  ✓ customerId → accountHolderKey (1.0)
  ✓ branch → assignedBranchKey (1.0)
  ✓ maturityDate → maturityDate (1.0)
  ✓ arrangementEffectiveDate → creationDate (0.9)
  ✓ currentBalance → totalBalance (0.9)
  ✓ accruedInterest → interestAccrued (1.0)
  ✓ category → accountType (0.9)
  ✓ interestRate → interestRate (1.0)
  ✓ interestStatement → interestPaymentPoint (1.0)
  ✓ status → status (0.9)
  📊 Added 14969 rows to Bank2_Mock_Deposit_Accounts (2)

🔄 Mapping: Bank1_Mock_CurSav_Accounts (2) → Bank2_Mock_Deposit_Accounts (2)
  ✓ accountId → encodedKey (0.95)
  ✓ displayName → name (0.9)
  ✓ productId → productTypeKey (1.0)
  ✓ currency → currencyCode (1.0)
  ✓ customerId → accountHolderKey (1.0)
  ✓ branch → assignedBr

...

  📊 Added 15071 rows to Bank2_Mock_Deposit_Accounts (2)

🔄 Mapping: Bank1_Mock_FixedTerm_Transactions (2) → Bank2_Mock_Deposit_Transactions (2)
  ✓ transactionReference → encodedKey (0.95)
  ✓ activity → type (1.0)
  ✓ transactionAmount → amount (1.0)
  ✓ currency → currencyCode (1.0)
  ✓ effectiveDate → valueDate (1.0)
  ✓ transactionDate → creationDate (0.9)
  ✓ reason → notes (0.8)
  ✓ branch → branchKey (1.0)
  ✓ chargesChargeAmount → feesAmount (1.0)
  ✓ interestAmount → interestAmount (1.0)
  ✓ taxRate → taxRate (1.0)
  ✓ currentBalance → totalBalance (0.9)
  ✓ interestRate → interestRate (1.0)
  ✓ channel → transactionChannel (1.0)
  ✓ accountId → parentAccountKey (1.0)


/tmp/ipython-input-1638667000.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_tables[target_table] = pd.concat(


  📊 Added 75265 rows to Bank2_Mock_Deposit_Transactions (2)

🔄 Mapping: Bank1_Mock_CurSav_Transactions (2) → Bank2_Mock_Deposit_Transactions (2)
  ✓ transactionReference → encodedKey (0.95)
  ✓ activity → type (1.0)
  ✓ transactionAmount → amount (1.0)
  ✓ currency → currencyCode (1.0)
  ✓ effectiveDate → valueDate (1.0)
  ✓ transactionDate → creationDate (0.9)
  ✓ bookingDate → bookingDate (1.0)
  ✓ narrative → notes (0.8)
  ✓ branch → branchKey (1.0)
  ✓ chargesChargeAmount → feesAmount (1.0)
  ✓ taxRate → taxRate (1.0)
  ✓ availableBalance → availableBalance (1.0)
  ✓ lockedAmount → lockedBalance (1.0)
  ✓ approvedOverdraftLimit → overdraftLimit (0.9)
  ✓ channel → transactionChannel (1.0)
  ✓ accountId → parentAccountKey (1.0)
  ✓ chargesPropertyName → feesName (0.9)


/tmp/ipython-input-1638667000.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_tables[target_table] = pd.concat(


  📊 Added 74604 rows to Bank2_Mock_Deposit_Transactions (2)

✅ MERGE COMPLETE!

📊 Final Table Sizes:
  Bank2_Mock_Loan_Transactions (2): 148234 rows (+73313 from Bundle 1)
  Bank2_Mock_Loan_Accounts (2): 29715 rows (+14729 from Bundle 1)
  Bank2_Mock_Identifications (2): 20000 rows (+10000 from Bundle 1)
  Bank2_Mock_Deposit_Transactions (2): 249016 rows (+149869 from Bundle 1)
  Bank2_Mock_Deposit_Accounts (2): 49796 rows (+30040 from Bundle 1)
  Bank2_Mock_Customer (2): 20000 rows (+10000 from Bundle 1)
  Bank2_Mock_Addresses (2): 20000 rows (+10000 from Bundle 1)


In [33]:
# Cell 6: Save & Download Results
# Save mapping documentation
with open('mapping_documentation.json', 'w') as f:
    json.dump(mappings, f, indent=2)

print("💾 Saving merged tables...")

# Save each table as CSV
output_files = []
for table_name, df in merged_tables.items():
    filename = f"merged_{table_name}.csv"
    df.to_csv(filename, index=False)
    output_files.append(filename)
    print(f"  ✓ Saved: {filename}")

# Create zip file with all outputs
print("\n📦 Creating output package...")
with zipfile.ZipFile('merged_output.zip', 'w') as zipf:
    zipf.write('mapping_documentation.json')
    for filename in output_files:
        zipf.write(filename)

print("\n📥 Download merged data:")
files.download('merged_output.zip')

print("\n✅ Done! Check the zip file for:")
print("  - All merged tables (CSV format)")
print("  - mapping_documentation.json (shows how data was mapped)")

💾 Saving merged tables...
  ✓ Saved: merged_Bank2_Mock_Loan_Transactions (2).csv
  ✓ Saved: merged_Bank2_Mock_Loan_Accounts (2).csv
  ✓ Saved: merged_Bank2_Mock_Identifications (2).csv
  ✓ Saved: merged_Bank2_Mock_Deposit_Transactions (2).csv
  ✓ Saved: merged_Bank2_Mock_Deposit_Accounts (2).csv
  ✓ Saved: merged_Bank2_Mock_Customer (2).csv
  ✓ Saved: merged_Bank2_Mock_Addresses (2).csv

📦 Creating output package...

📥 Download merged data:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Done! Check the zip file for:
  - All merged tables (CSV format)
  - mapping_documentation.json (shows how data was mapped)
